In [15]:
import numpy as np
import scipy
import statsmodels
import pandas as pd

# Решение тренировочного теста по критерям Стьюдента

1.  *1-stats.t.cdf(t, nu);*
2.  *Признаки устроены так, что mu1 < mu2 в принципе невозможно;*
3.  *Приближенное решение точно при n1=n2; Проблема заключается в том, что не существует точного способа сравнить среднее двух выборок с неизвестными дисперсиями;*

**Вопрос 4**

Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

Округлите ответ до четырёх знаков после десятичной точки.

известны дисперсии, значит Z-критерий.

нулевая гипотеза: mu0 = 9.5
альтернатива: m0 != 9.5   -> двусторонняя

Z = (mean(X) - mu0) / (sigma/sqrt(n))

p-value = 2 ( 1 - F(|z|))

In [2]:
z = (9.57 - 9.5)/(0.4/np.sqrt(160))
z

2.213594362117875

In [6]:
round(2*(1-stats.norm.cdf(abs(z))), 4)

0.0269

**Вопрос 6**

Отделите 25% случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На обучающей выборке настройте две регрессионные модели:

- линейную регрессию с помощью LinearRegression без параметров

- случайный лес из 10 деревьев с помощью RandomForestRegressor с random_state=1.

Какая из моделей лучше предсказывает цену бриллиантов? Сделайте предсказания на тестовой выборке, посчитайте модули отклонений предсказаний от истинных цен. Проверьте гипотезу об одинаковом среднем качестве предсказаний, вычислите достигаемый уровень значимости. Отвергается ли гипотеза об одинаковом качестве моделей против двусторонней альтернативы на уровне значимости alpha=0.05?

In [2]:
data = pd.read_csv('diamonds.txt', sep = '\t', header = 0)
data

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,61.0,58.0,2757,6.15,6.12,3.74


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['price']), data.price, test_size=0.25, random_state=1)

In [12]:
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)
predictions_lr = linear_regression.predict(X_test)

In [13]:
from sklearn.ensemble import RandomForestRegressor

randomforest_regression = RandomForestRegressor(n_estimators=10, random_state=1)
randomforest_regression.fit(X_train,y_train)
predictions_rfr = randomforest_regression.predict(X_test)

In [14]:
stats.ttest_rel(abs(y_test - predictions_lr), 
                abs(y_test - predictions_rfr))

Ttest_relResult(statistic=13.017729783878393, pvalue=1.655174575144972e-38)

259306.23929491662

**Вопрос 7**

В предыдущей задаче посчитайте 95% доверительный интервал для разности средних абсолютных ошибок предсказаний регрессии и случайного леса. Чему равна его ближайшая к нулю граница? Округлите до десятков (поскольку случайный лес может давать немного разные предсказания в зависимости от версий библиотек, мы просим вас так сильно округлить, чтобы полученное значение наверняка совпало с нашим).

In [28]:
from statsmodels.stats.weightstats import *

DescrStatsW(abs(predictions_lr - y_test) - abs(predictions_rfr - y_test)).tconfint_mean()

(74.2872453259534, 100.62452098634195)